In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

sport = pd.read_excel('data_files/Days.xlsx',sheet_name='health')

In [10]:
sport = sport.loc[:,['DATE', 'SPORT_TIME']]

In [31]:
sport = sport.dropna()
sport = sport.tail(50)
sport = sport.reset_index()

In [32]:
sport

,DATE,SPORT_TIME
0,2023-01-22,32.0
1,2023-01-23,33.0
2,2023-01-24,55.0
3,2023-01-25,33.0
4,2023-01-26,36.0
5,2023-01-27,45.0
6,2023-01-28,140.0
7,2023-01-29,67.0
8,2023-01-30,36.0
9,2023-01-31,20.0


In [33]:
# Make sure the 'DATE' column is in datetime format
sport['DATE'] = pd.to_datetime(sport['DATE'])

# Set the 'DATE' column as the index
sport = sport.set_index('DATE')

# Resample the dataframe to daily frequency, filling missing dates with zeros
df_daily = sport.resample('D').sum().fillna(0)

In [34]:
# Create a function to normalize the Sport values and map them to a colormap
def map_activity_to_color(value, cmap, vmin, vmax):
    normalized_value = (value - vmin) / (vmax - vmin)
    return cmap(normalized_value)

In [35]:
# Create the diagram with small squares

def create_activity_diagram(df, figsize=(10, 10), cmap=plt.cm.Blues, vmax=None, output_file=None):
    # Find the date range and create an empty array
    days = (df.index.max() - df.index.min()).days + 1
    weeks = int(np.ceil(days / 7))
    data = np.zeros((weeks, 7))

    # Fill the data array with Sport values
    start_date = df.index.min()
    for i in range(weeks):
        for j in range(7):
            current_date = start_date + timedelta(weeks=i, days=j)
            if current_date in df.index:
                data[i, j] = df.loc[current_date, 'SPORT_TIME']

    if vmax is None:
        vmax = data.max()

    # Create the diagram
    fig, ax = plt.subplots(figsize=figsize)
    for i in range(weeks):
        for j in range(7):
            value = data[i, j]
            color = map_activity_to_color(value, cmap, 0, vmax)
            rect = plt.Rectangle((j, i), 1, 1, facecolor=color, edgecolor='black')
            ax.add_patch(rect)

    ax.set_xlim(0, 7)
    ax.set_ylim(0, weeks)
    ax.set_yticks(np.arange(0, weeks) + 0.5)
    ax.set_yticklabels([start_date + timedelta(weeks=i) for i in range(weeks)], rotation=90, ha='right')
    ax.set_xticks(np.arange(0, 7) + 0.5)
    ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    ax.invert_yaxis()

    if output_file:
        plt.savefig(output_file, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()


# Create the diagram
create_activity_diagram(df_daily, output_file='activity_diagram.jpg')
